In [39]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
import geopandas as gpd
import xarray as xr
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
from scipy.stats import entropy
import cartopy.feature as cf

In [40]:
shp_pan = gpd.read_file(r'Data\amazon\panamz.geojson')
import cartopy.crs as ccrs
import cartopy.feature as cf

In [41]:
shp_pan = gpd.read_file(r'Data\amazon\panamz.geojson')
#stations geodataframe
gdf = gpd.read_file(r'Data\all_stations.geojson')
# rainfall cleaned data
df_bet= pd.read_pickle('Data\ALL_01_20_bet.pkl')
gdf_filtered = gdf[gdf.Code.isin(df_bet.Code.unique())].to_crs(shp_pan.crs)
gdf_pan = gpd.sjoin(gdf_filtered, shp_pan, how='inner', op='intersects')

c:\Users\luisg\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


## test plot months

In [11]:
# get points from gdf_filtered that are inside shp
gdf_amz = gdf_pan
df_amz =  df_bet[df_bet.Code.isin(gdf_amz.Code.unique())]
#df_amz to pkl
#df_amz.to_pickle(r'Data\amazon\df_amz_bet.pkl')

In [94]:
df_clima = df_amz.groupby(['Code', df_amz.Date.dt.month]).mean()
df_clima = df_clima.unstack()
df_clima.columns = df_clima.columns.droplevel(0)
df_clima.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
df_clima = df_clima.join(gdf_amz.set_index('Code')['geometry'])
df_clima = gpd.GeoDataFrame(df_clima)

C:\Users\luisg\AppData\Local\Temp\ipykernel_18884\152352569.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clima = df_amz.groupby(['Code', df_amz.Date.dt.month]).mean()


In [95]:
vmax = 400
vmin = 0
#df_clima[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']].max().max()
#df_clima[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']].min().min()
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# fake up the array of the scalar mappable. Urgh...
sm._A = []

In [96]:
##plot for each month of df_clima
#fig, axes = plt.subplots(4, 3, figsize=(10,15))
#axes = axes.flatten()
#cax = fig.add_axes([1, 0.1, 0.03, 0.8])
#fig.colorbar(sm, cax=cax, extend='both')
#
#for i, ax in enumerate(axes):
#    shp_amz.plot(ax = ax, color='white', edgecolor='black')
#    df_clima.plot(column=df_clima.columns[i], ax=ax, vmin=vmin, vmax=vmax, markersize=10)
#    ax.set_title(df_clima.columns[i])
#

# metrics

In [4]:
gdf_amz = gdf_amz.set_index('Code').sort_index()
gdf_amz['MAP'] = (df_amz.groupby(['Code']).mean()*12).sort_index()
gdf_amz = gdf_amz[['Name', 'Latitude', 'Longitude', 'Altitude', 'geometry','MAP']]
#gdf_amz.to_file(r'Data\amazon\gdf_amz_metrics.geojson', driver='GeoJSON')

C:\Users\luisg\AppData\Local\Temp\ipykernel_19900\200304027.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gdf_amz['MAP'] = (df_amz.groupby(['Code']).mean()*12).sort_index()


In [5]:
df_dry = df_amz[df_amz['Total']<100]
gdf_amz['DSL'] = df_dry.groupby(['Code', df_dry.Date.dt.year]).size().groupby('Code').mean()
gdf_amz['DSL'] = gdf_amz['DSL'].fillna(0)

In [6]:
#calculate Relative Entropy
df_amz = df_amz.sort_values(by=['Code', 'Date']).reset_index(drop=True)
df_amz['ratio'] = df_amz['Total'] / df_amz.groupby(['Code', df_amz.Date.dt.year]).transform(np.sum)['Total']
uniform = np.zeros(12)
uniform[:] = 1/12
# aplly the entropy function to the ratio column for each code
df_amz_ent_year = df_amz.groupby(['Code', df_amz.Date.dt.year])['ratio'].apply(lambda x: entropy(x, qk=uniform,base=2)).reset_index().rename({'ratio': 'Entropy'},  axis='columns')
ent_y = df_amz_ent_year.groupby('Code')['Entropy'].mean().reset_index()
gdf_amz['Entropy'] = ent_y.set_index('Code')['Entropy']

C:\Users\luisg\AppData\Local\Temp\ipykernel_19900\2058043839.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_amz['ratio'] = df_amz['Total'] / df_amz.groupby(['Code', df_amz.Date.dt.year]).transform(np.sum)['Total']


In [9]:
# seasonal climatology
df_amz['month'] = df_amz.Date.dt.month
df_amz['season'] = df_amz['month'].apply(lambda x: 'DJF' if x in [12,1,2] else 'MAM' if x in [3,4,5] else 'JJA' if x in [6,7,8] else 'SON')


In [13]:
df_sea = df_amz.groupby(['Code', 'season']).mean().reset_index().pivot(index='Code', columns='season', values='Total')

C:\Users\luisg\AppData\Local\Temp\ipykernel_19900\223084753.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_sea = df_amz.groupby(['Code', 'season']).mean().reset_index().pivot(index='Code', columns='season', values='Total')


In [15]:
#merge to gdf_amz
gdf_amz = gdf_amz.join(df_sea)

In [16]:
#gdf_amz.to_file(r'Data\amazon\gdf_amz_metrics.geojson', driver='GeoJSON')

# seasonal

## all stations